---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
for j in range(len(P1_Graphs)):
        G = P1_Graphs[j]
        print(G)
#1,4 barabasi_albert_graph so it's PA
#2,3,5 watts_strogatz_graph but p in 1,2 less than <.1 and with high value 1 in 4

   

barabasi_albert_graph(1000,2)
watts_strogatz_graph(1000,10,0.05)
watts_strogatz_graph(750,5,0.075)
barabasi_albert_graph(750,4)
watts_strogatz_graph(750,4,1)


In [5]:
def graph_identification():
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
graph_identification()            

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [7]:
G.nodes(data=True)

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0}),
 (10, {'Department': 9, 'ManagementSalary': 0.0}),
 (11, {'Department': 14, 'ManagementSalary': 0.0}),
 (12, {'Department': 14, 'ManagementSalary': 1.0}),
 (13, {'Department': 26, 'ManagementSalary': 1.0}),
 (14, {'Department': 4, 'ManagementSalary': nan}),
 (15, {'Department': 17, 'ManagementSalary': 0.0}),
 (16, {'Department': 34, 'ManagementSalary': 0.0}),
 (17, {'Department': 1, 'ManagementSalary': 0.0}),
 (18, {'Department': 1, 'ManagementSalary': nan}),
 (19, {'Department': 14, 'Ma

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
def salary_predictions():
    df= pd.DataFrame(index=G.nodes())
    df['is_management'] =  pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G,normalized=True))
    df['PR'] = pd.Series(nx.pagerank(G))  
    features = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweenness', 'PR']
    df_train = df[~pd.isnull(df['is_management'])]
    df_test  = df[pd.isnull(df['is_management'])]
    X_train =  df_train[features]
    X_test  =  df_test[features]
    y_train =  df_train['is_management']
    #X_test  =  df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = RandomForestClassifier(n_estimators = 100,max_depth=10 ,random_state=0).fit(X_train_scaled, y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1].tolist()
    return pd.Series(test_proba,X_test.index)
salary_predictions()

1       0.011184
2       0.970000
5       1.000000
8       0.302178
14      0.054863
18      0.092933
27      0.034835
30      0.773090
31      0.427759
34      0.113333
37      0.075220
40      0.028968
45      0.004164
54      0.404611
55      0.492629
60      0.289923
62      1.000000
65      0.990000
77      0.001881
79      0.066888
97      0.020000
101     0.010220
103     0.534776
108     0.034243
113     0.053165
122     0.010526
141     0.281966
142     1.000000
144     0.000556
145     0.563968
150     0.353756
154     0.854950
158     0.138288
175     0.000000
181     0.098753
193     0.268193
196     0.099941
200     0.112949
202     0.000000
204     0.257648
214     0.019322
215     0.935000
230     0.153946
231     0.802500
236     0.759936
239     0.000556
240     0.010167
247     0.010000
249     1.000000
251     0.598045
252     0.920000
253     0.258466
255     0.665468
261     0.740851
262     0.001314
264     0.348963
266     0.020934
267     0.010000
270     0.2405

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [21]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    df['common_neigh'] = [len(list(nx.common_neighbors(G,x[0],x[1]))) for x in nx.non_edges(G)]
    df['jaccard_coef'] = [x[2] for x in list(nx.jaccard_coefficient(G)) ]   
    df['resource_all'] = [x[2] for x in list(nx.resource_allocation_index(G)) ]
    df['adamic_adar' ] = [x[2] for x in list (nx.adamic_adar_index(G))]
    df['cn_sound_hop'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G, community='Department'))]
    df["ra_sound_hop"] = [x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G, community='Department'))]
    df["with_E_clust"] = [x[2] for x in list(nx.within_inter_cluster(G, community='Department',delta=0.5))]
    features = ['preferential_attachment','common_neigh','resource_all','adamic_adar','cn_sound_hop','cn_sound_hop',
                'ra_sound_hop','with_E_clust']
    df = df.merge(future_connections, left_index=True, right_index=True)
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test  = df[pd.isnull(df['Future Connection'])]
    X_train =  df_train[features]
    X_test  =  df_test[features]
    y_train =  df_train['Future Connection']
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = RandomForestClassifier(n_estimators = 100,max_depth=10 ,random_state=0).fit(X_train_scaled, y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    return pd.Series(test_proba,X_test.index)
new_connections_predictions()